https://towardsdatascience.com/cifar-100-transfer-learning-using-efficientnet-ed3ed7b89af2

In [1]:
import os
import cv2
import glob
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from keras.utils import load_img
from keras.utils import img_to_array
from keras.models import load_model
from keras.metrics import top_k_categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator

2023-06-23 10:16:31.496159: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
CATEGORIES_PATH = "categories.csv"
CATEGORY_PATH = "dataset_sam/category_id.txt"
TEST_PATH = "dataset/test/"
TEST_TOP10_PATH = "dataset/test_top10/"
TEST_LAST10_PATH = "dataset/test_last10/"
TEST_SAM_PATH = "dataset_sam/test/"
TEST_DIRS = glob.glob("dataset/test/*")
TEST_TOP10_DIRS = glob.glob("dataset/test_top10/*")
TEST_LAST10_DIRS = glob.glob("dataset/test_last10/*")

MODELS = ["food-seg-103-xception", "food-seg-103-densenet121", "food-seg-103-densenet201"]
CHECKPOINT_PATHS = ["checkpoints/" + MODEL + ".h5" for MODEL in MODELS]
MODEL_PATHS = ["models/" + MODEL + ".h5" for MODEL in MODELS]

IMAGE_SIZE = 512

In [3]:
categories = list(pd.read_csv(CATEGORIES_PATH, header=None)[0])

In [4]:
def get_classes():
    cats = pd.read_csv(CATEGORY_PATH, sep="\t", names=["id", "name"])
    ids = cats["id"].to_list()
    classes = cats["name"].to_list()
    classes = [cls.strip() for cls in classes]
    return ids, classes

In [5]:
def acc_top5(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=5)

In [6]:
def load_models():
    loaded_models = []
    for CHECKPOINT_PATH in CHECKPOINT_PATHS:
        loaded_models.append(load_model(CHECKPOINT_PATH, custom_objects={"acc_top5": acc_top5}))
    print("Models Loaded")
    return loaded_models

In [7]:
models = load_models()

Models Loaded


In [8]:
def get_label(row, reduction="mean"):
    max_freq_predictions = []
    predictions = row.iloc[:len(MODELS)]
    label_counts = row.iloc[len(MODELS):]
    max_freq_labels = label_counts[label_counts == label_counts.max()].index.tolist()
    for max_freq_label in max_freq_labels:
        scores = []
        for prediction in predictions:
            label, score = prediction
            if label == max_freq_label:
                scores.append(score)
        if reduction == "mean":
            max_freq_score = 0 if len(scores) == 0 else sum(scores) / len(scores)
        else:
            max_freq_score = 0 if len(scores) == 0 else max(scores)
        max_freq_predictions.append([max_freq_label, max_freq_score])
    max_freq_prediction = max(max_freq_predictions, key=lambda item: item[1])
    return {max_freq_prediction[0]: max_freq_prediction[1]}

In [9]:
def predict(model, filepath, top_n=None):
    test_image = load_img(filepath, target_size=(IMAGE_SIZE, IMAGE_SIZE))
    test_image_array = img_to_array(test_image)
    test_image_array = np.expand_dims(test_image_array, axis=0)
    test_image_array = test_image_array / 255.
    predictions = model.predict(test_image_array, verbose=0)
    if top_n:
        predicted_labels = np.argsort(predictions[0])[::-1][:top_n]
    else:
        predicted_labels = np.argsort(predictions[0])[::-1]
    predicted_scores = predictions[0][predicted_labels]
    return predicted_labels, predicted_scores

In [10]:
def ensemble_predict(filepath, top_n_predictions=None, ensemble_type=1, reduction="mean"):
    """
        Ensemble prediction allow filepath or base64, and contains 2 types:

        Parameters:
        ensemble_type (int): 1 or 2.
            For 1: top_n_predictions is available.
            For 2: reduction is available.
        reduction (str): mean or max
    """
    if ensemble_type == 1:
        predictions = {}
        for model in models:
            predicted_labels, predicted_scores = predict(model, filepath)
            for i, label in enumerate(predicted_labels):
                if categories[predicted_labels[i]] in predictions:
                    if predictions[categories[predicted_labels[i]]] < predicted_scores[i]:
                        predictions[categories[predicted_labels[i]]] = predicted_scores[i]
                else:
                    predictions[categories[predicted_labels[i]]] = predicted_scores[i]
        if top_n_predictions:
            predictions = dict(sorted(predictions.items(), key=lambda item: item[1], reverse=True)[:top_n_predictions])
        else:
            predictions = dict(sorted(predictions.items(), key=lambda item: item[1], reverse=True))
    else:
        predictions = {}
        for i, model in enumerate(models):
            predicted_labels, predicted_scores = predict(model, filepath, top_n=1)
            predictions[MODELS[i]] = [[categories[predicted_labels[0]], predicted_scores[0]]]
        predictions = pd.DataFrame(predictions)
        predictions.columns = ["Prediction" + str(i + 1) for i in range(predictions.shape[1])]
        label_counts = predictions.apply(lambda row: pd.Series([item[0] for item in row]).value_counts(), axis=1).fillna(0)
        predictions = pd.concat([predictions, label_counts], axis=1)
        predictions = list(predictions.apply(lambda row: get_label(row, reduction), axis=1))[0]
    return predictions

In [11]:
test_size = 0
for test_dir in TEST_DIRS:
    name = os.path.basename(test_dir)
    for image in glob.glob(TEST_PATH + name + "/*"):
        test_size += 1

running_correct = 0
running_size = 0
with tqdm(total=test_size) as pbar:
    for test_dir in TEST_DIRS:
        name = os.path.basename(test_dir)
        for img_path in glob.glob(TEST_PATH + name + "/*"):
            predictions = ensemble_predict(img_path, top_n_predictions=1, ensemble_type=2, reduction="mean")
            predicted_name = list(predictions.keys())[0]
            if predicted_name == name:
                running_correct += 1
            running_size += 1
            pbar.set_postfix({"accuracy": float(running_correct) / running_size})
            pbar.update(1)

100%|█████████████████████| 9832/9832 [7:04:35<00:00,  2.59s/it, accuracy=0.679]


In [12]:
def convert_channels_to_labels(filepath):
    labels = []
    ids, classes = get_classes()
    unique_channels = []
    image = cv2.imread(filepath)
    num_channels = image.shape[2]
    for i in range(num_channels):
        channel = image[:, :, i]
        unique_values = np.unique(channel)
        unique_channels = unique_channels + list(unique_values)
        unique_channels = list(np.unique(np.asarray(unique_channels)))
    for channel in unique_channels:
        if channel != 0:
            labels.append(classes[channel])
    return labels

In [13]:
def calculate_f_score(predicted_labels, true_labels, beta=1):
    predicted_labels = set(predicted_labels)
    true_labels = set(true_labels)
    tp = len(predicted_labels.intersection(true_labels))
    fp = len(predicted_labels - true_labels)
    fn = len(true_labels - predicted_labels)
    tn = len(predicted_labels.union(true_labels)) - tp - fp - fn
    precision = tp / (tp + fp + 1e-12)
    recall = tp / (tp + fn + 1e-12)
    f_score = (1 + beta**2) * precision * recall / (beta**2 * precision + recall + 1e-12)
    return f_score, precision, recall

In [16]:
test_size = 0
for f in os.listdir(TEST_SAM_PATH + "img"):
    f_path = os.path.join(TEST_SAM_PATH + "img", f)
    if os.path.isdir(f_path):
        test_size += 1

threshold = 0.7
f_scores = []
precisions = []
recalls = []
with tqdm(total=test_size) as pbar:
    for f in os.listdir(TEST_SAM_PATH + "img"):
        f_path = os.path.join(TEST_SAM_PATH + "img", f)
        if os.path.isdir(f_path):
            true_labels = convert_channels_to_labels(TEST_SAM_PATH + "mask/" + os.path.basename(f_path) + ".png")
            predicted_labels = []
            for image_name in os.listdir(f_path):
                if "mask" not in image_name:
                    img_path = os.path.join(f_path, image_name)
                    predictions = ensemble_predict(img_path, top_n_predictions=1, ensemble_type=2, reduction="mean")
                    predicted_name = list(predictions.keys())[0]
                    predicted_score = list(predictions.values())[0]
                    if predicted_score > threshold and predicted_name != "background":
                        predicted_labels.append(predicted_name)
                    predicted_labels = list(set(predicted_labels)) 
            f_score, precision, recall = calculate_f_score(predicted_labels, true_labels)
            f_scores.append(f_score)
            precisions.append(precision)
            recalls.append(recall)
            pbar.set_postfix({"f": sum(f_scores) / len(f_scores),
                              "p": sum(precisions) / len(precisions),
                              "r": sum(recalls) / len(recalls)})
            pbar.update(1)
print("f_score:", (sum(f_scores) / len(f_scores)),
      "precision:", (sum(precisions) / len(precisions)),
      "recall:", (sum(recalls) / len(recalls)))

100%|███████████████| 105/105 [47:06<00:00, 26.92s/it, f=0.364, p=0.49, r=0.328]

f_score: 0.36378066378018736 precision: 0.4902721088432567 recall: 0.3282766439908135
